# Tutorial : Combinatorial sequences with SageMath
## July 25, 2023 - Arusha, Tanzania

### Author: Nadia Lafrenière

The goal of this sequence of exercises is to put in practice a few different approaches to combinatorial structures with SageMath.

Acknowledgements: I'd like to thank Per Alexandersson for suggesting some ideas explored here.


## Stirling numbers

**Question** 37 from Per Alexandersson's notes
> How many permutations in $S_{10}$ are there of type $(2, 2, 2, 2, 1, 1)$?

This can be solved through many ways: 
- Multinomial coefficients, as you learned in the course
- Brute force: generating permutations of 10, filtering, then counting

In [1]:
# Using multinomial coefficients. If you don't know how to get multinomial coefficients, search the documentation or
# ask another student


4725

In [2]:
# Generating permutations of 10, filtering, counting.
# WARNING: you'll need to be clever to do it without making SageMath crash.
# If this takes too long, interrupt your computations with the square above, and try to 
#g et the number of permutations  type. That will be handy later


4725

In [1]:
# Another technique that we will explore in more details later this week is to gather information for many
# instances (here partitions), and then to supply this information to a database.

data = []
for n in range(1,5):
    for p in Partitions(n):
        count = 0
        for sigma in Permutations(n):
            if sigma.cycle_type() == p:
                count += 1
        data.append((p, count))
dict(data)

{[1]: 1,
 [2]: 1,
 [1, 1]: 1,
 [3]: 2,
 [2, 1]: 3,
 [1, 1, 1]: 1,
 [4]: 6,
 [3, 1]: 8,
 [2, 2]: 3,
 [2, 1, 1]: 6,
 [1, 1, 1, 1]: 1}

The code above is very naive, but it still works because the number of objects is small.

Later this week, we will see a process for finding out the name of this statistic, using the FindStat database. Here is a preview.

In [11]:
findstat(data, depth=0)

0: St000182 (quality [100, 100])

In [8]:
findstat(182)

St000182: The number of permutations whose cycle type is the given integer partition.

Looking at the documentation for Statistics with Id 182 in FindStat, we see that there is a product formula to count them. It is worth having a look (to do so, execute the next cell).

In [9]:
_.browse()

Another potential, less naive methods for generating all permutations with a given cycle type:
>T = SetPartitions(10, [2,2,2,2,1,1]) and then T.random_element() to get a partition into cycles. Because all cycles have length at most 2, there is a unique such permutation for each set partition. Then form all possible permutations from them.

(from https://ask.sagemath.org/question/65102/is-there-a-way-in-sage-to-generate-a-random-permutation-with-a-given-cycle-type/)

Applying `SetPartitions` along with cardinality, we get the right answer.

4725

We could also inspect `cardinality` to see how this is calculated:

## Sieving
**Question** 55 from Per Alexandersson's notes:
> How many numbers from {1, . . . , 100} are divisible by either 3, 5 or 11?

This is also the number obtained using inclusion-exclusion.

## Number of derangements

**Question**: Secret Santa
> For the end of the ICTP-EAUMP school, we will organize a gift exchange. We will put a bowl with all the names in it, and ask every participant to draw one name. The exchange is *successful* if no one draws their own name. What are the odds, roughly, that this happens?

In the previous lectures, you learned how to do it using inclusion-exclusion, and you might have learned how to find the asymptotics. What I'm suggesting here is a method for approximating the answer, done through sampling.

Idea: simulating 1,000 (or more, or less) gift exchanges (uniformly at random), and see what proportion is successful.

- Generating 1,000 permutations of 40, and checking if they have fixed points = **EASY**
- Generating all 40! permutations of 40 and checking if they have fixed points = **IMPOSSIBLE**

This method has the advantage of not requiring knowledge about derangements (permutations without fixed points).

In [3]:
# Find a way to generate 1,000 permutations of 40, at random. Count the number of successful exchanges.

377

Hence, we would estimate that around 35-40% are successful. The actual proportion is $e^{-1}$.

- Finding the exact answer requires a proof, and knowledge. Finding exact asymptotics too.
- Simulation allows one to get a very good rough idea of how big a quantity is.

## Eulerian numbers

Recall from Per Alexandersson's lecture that the Eulerian number $A(n,k)$ is the number of permutations of $S_n$ with exactly $k$ descents.
Recall that for $\pi \in S_n$, a *descent* in the permutation $\pi$ is an index $i \in [n − 1]$ such that $\pi(i) > \pi(i + 1)$.

> Write some code to compute the Eulerian number $A(10,4)$.

In [9]:
# Write a function called 'my_eulerian_number(n, k)' that computes A(n, k)


In [ ]:
n = 10
k = 4
%time my_eulerian_number(n, k)

A method called 'eulerian_number' exists in sage.combinat.combinat. Import the appropriate module (like we did in the lecture for the Narayana numbers), then evaluate how much time it takes to execute it (for $n=10$ and $k=4$ again). We will use it to compare our results.

In [4]:
# Your answer here

We see that we get the same number, but the built-in methods is much faster. This is a hint that we should look at the code for `eulerian_number`

In [12]:
eulerian_number??

We see that it uses a recursion. In Per Alexandersson's course, you learned that
$$A(n, k) = (n − k)A(n − 1, k − 1) + (k + 1)A(n − 1, k).$$

Now use it to write a fast method for eulerian numbers. It might have to be recursive (so calling the same method over a smaller instance).

In [14]:
# Write a function called `my_eulerian_number_fast(n, k)`.

In [ ]:
%time my_eulerian_number_fast(10, 4)

However, this methods gets pretty slow when $n$ and $k$ grow, especially in comparison with eulerian_number. This is because it computes **many** times the same things. Execute the two cells below to see the time difference.

In [ ]:
%time my_eulerian_number_fast(30, 7)

In [ ]:
%time eulerian_number(30, 7)

We can avoid this by *caching* the result of our previous computations. This will require a little more memory, but so much less time. This is achieved through a *decorator*, so something we put right before the start of the function.

Write a cached function, by doing the same thing as before, but adding `@cached_function` on the line before the definition.

In [20]:
# Your answer here

In [ ]:
%time my_eulerian_number_fast(30, 7)

**Bonus exercise:** Come up with another combinatorial calculation where the use of the decorator `@cached_function` could be helptul. Test your hypothesis by writing the code for it, both with and without decorator.